<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Flexible, Scalable Storage and Analytics with VantageCloud Lake
  <br>
       <img id="teradata-logo" src="images/TeradataLogo.png" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<br>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Teradata's VantageCloud Lake Edition is the only enterprise-class analytics platform that enables powerful, open, and flexible analytics against data in any location; Data Lakes, third-party systems, and traditional EDW.</p>

<hr>

<b style = 'font-size:28px;font-family:Arial;color:#00233C'>Environment Overview</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This demonstration will illustrate how users can can use Vantage to access data from a variety of external sources, choose from multiple physical and virtual storage tiers, and then derive valuable analytic and business insights at extreme scale with minimal cost.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Data Architects and Engineers have a choice of a broad data locality and storage types that can help optimize the balance between cost, accessibility, and analytic performance.  These storage types include:</p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top;font-size:16px;font-family:Arial;color:#00233C' width = '40%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Direct-attached block storage for high-speed, tactical workloads.  Most common for smaller, dimensional tables, or tables that support tactical, near-real-time query patterns.</li>
                <br>
                <li>An object-storage-based filesystem named the Vantage Object Filesystem (OFS) which features:
                    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
                        <li>Massively scalable low-cost storage</li>
                        <li>Data sharing across multiple processing clusters</li>
                <li>Full ACID compliance and optimized for performance including caching, optimized spool, indexed metadata</li>
                    </ul>
                </li>
                <br>
                <li>Open Object Storage; which can include Data Lakes (including formats such as Delta), IIOT data, Data Marketplaces, and many other third-party ETL and data streaming systems.</li>
            </ol>
        </td>
        <td><img src = 'images/Lake_Architecture.png' width = '800'></td>
    </tr>
</table>

<b style = 'font-size:28px;font-family:Arial;color:#00233C'>Demonstration Overview</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This notebook consists of three primary demonstrations designed to briefly review these storage types and how they can be mixed transparently for analytical processing.  </p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Demo 1 - The Object Filesystem (OFS)</b>. Show how users can load data, execute queries, as well as how leverage Time Travel capabilities</li>
    <li><b>Demo 2 - Open Object Storage</b> - Data Discovery and analytics against data in S3, and an illustration of how to simplify access for users</li>
    <li><b>Demo 3 - Analytics</b> - Combine data across storage tiers and apply powerful, scalable ClearScape Analytics Functions</li>
    </ol>


<hr>

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Connect to Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This notebook uses the Vantage Plugin for Jupyter; a Jupyter Lab Extension that provides native SQL processing, administrative magic functions, and visualiztion capabilities.  In order to conntect to our Vantage system, the user needs to create a connection - this connection is stored in the user's profile and doesn't need to be recreated each time.</p> 

In [ ]:
%addconnect name=lake_demo, host=XXX.XXX.XXX.XXX

In [ ]:
%chconnect name=lake_demo, host=XXX.XXX.XXX.XXX

In [ ]:
%disconnect name=lake_demo

In [ ]:
%connect lake_demo, user=data_engineer, hidewarnings=True

<hr>
<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Demonstration 1 - Vantage Object FileSystem (OFS)</b></p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '40%'>
            <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Immutable storage, objects are versioned to guarantee consistency</li>
                <br>
                <li>The Object File System (OFS) index is represented by a tree structure with a single root with multiple leaf nodes:
                    <br>
                    <ul><br>
                        <li>One internal index structure is created for each OFS table</li>
                    <br>
                        <li>Internal indexes provide efficiency in multiple ways:<ul>
                            <li>Filtering out selected rows</li>
                            <li>Pre-positioning index entries across AMPs</li>
                            <li>Join column redistributions</li>
                            </ul></li>
                        <br>
                        <li>Set of statistics values for up to 64 columns carried in the root (for each leaf) and the leaf objects (for each data object)</li>
                    <br>
                <li>Each Root and Leaf node are in their own object, separate from the data objects</li>
                        <br>
                <li>With Columnar format, each Data Object contains one Row Group broken into Pages within Column Chunks</li></ul></li>
            </ul>
        </td>
        <td><img src = 'images/OFS_Columnar.png' width = '600'></td>
    </tr>
</table>

<hr>

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Create the OFS Table</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For purposes of demonstration, the user will DROP the table prior to loading so as to show how a new table can be created.</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Tables can be defined as generally normally done with any RDBMS.  However, in this case we will show some specifics that are unique to Vantage.</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li><b>First</b>, the "STORAGE" clause is set to indicate the data will be stored in the OFS framework.</li>
    <li><b>Second</b>, the SELECT that is used to populate the table consists of a specific query type designed to read data directly from an Object Store.</li> </ul>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Finally, the table can be queried as any normal relational table.</p>

In [ ]:
DROP TABLE demo.Txn_History_OFS

In [ ]:
/* Load the data into OFS directly from S3
*/

--EXPLAIN
CREATE TABLE demo.Txn_History_OFS,

    /*Note the use of STORAGE here to specify OFS
    This can also be set to a user and database default */
    
    STORAGE=TD_OFSSTORAGE
    
    /* We can use a SELECT statement to populate the table
    using WITH DATA */
    AS (

        /*Provide json path information to identify the proper fields */
        SELECT Payload.txn_id txn_id,
            Payload.step step,
            Payload."type" "txn_type",
            CAST(Payload.amount AS FLOAT) amount,
            Payload.nameOrig nameOrig,
            CAST(Payload.oldbalanceOrig AS FLOAT) oldbalanceOrig,
            CAST(Payload.newbalanceOrig AS FLOAT) newbalanceOrig,
            Payload.nameDest nameDest,
            CAST(Payload.oldbalanceDest AS FLOAT) oldbalanceDest,
            CAST(Payload.newbalanceDest AS FLOAT) newbalanceDest,
            CAST(Payload.isFraud AS INTEGER) isFraud,
            CAST(Payload.isFlaggedFraud AS INTEGER) isFlaggedFraud

        FROM (
            LOCATION = '/s3/s3.amazonaws.com/trial-datasets/FraudReduction/'
            AUTHORIZATION = Repositories.PubAuth
) AS D
) WITH DATA;

In [ ]:
/* Run a basic query against the data 
Note we can use AS OF syntax to select a specific version
Visualize results */

SELECT txn_type, COUNT(isFraud), SUM(isFraud) fraud_txns
FROM demo.Txn_History_OFS 

GROUP BY 1;

In [ ]:
%chart x=txn_type, y=fraud_txns, title="Count of Fraudulent Transactions"

<hr>
<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Time Travel with OFS</b></p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '40%'>
            <ul style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Benefits of Time Travel</b>
                <br>
                <br>
                <li>Querying the database based on how the data looked at some point in the past</li>
                <br>
                <li>If there were problems with a data load, users can go back to a point in time where the data is clean/consistent</li>
                 <br>
                 <li>Analytics can be re-run it keeping the inputs the same for one or more tables at any point in time</li>
                <br>
            </ul>
            <ul style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Features</b>
                <br>
                <br>
                <li>supported through SELECT AS OF syntax</li>
                <br>
                <li>Versions come with a system default retention period</li>
                <br>
                <li>Versioning allows readers to not be blocked by writers, and vice versa</li>
                <br>
                <li>A transactionally consistent read is always provided by using the last committed version</li>
            </ul>               
                </td>
        <td><img src = 'images/Time_Travel.png' width = '400'></td>
    </tr>
</table>

<hr>

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Time Travel Demonstration</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As discussed above, OFS tables have a defined retention period over which versions are stored.  This can be viewed by executing a SHOW STORAGE ACCESS statement to see the internal bucket information, including retention period.</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next, some additional data will be inserted into the table.  Executing a query gainst the VersionInfo view can show the version history</p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Finally, the table can be queried using AS OF syntax to illustrate simple row counts before and after changes</p>

In [ ]:
--Show the internal S3 bucket and retention (time travel) policy
SHOW STORAGE ACCESS TD_OFSSTORAGE;

In [ ]:
/* Insert some additional rows into our OFS Table */
INSERT INTO demo.Txn_History_OFS

        /*Provide json path information to identify the proper fields */
        SELECT TOP 10000 Payload.txn_id txn_id,
            Payload.step step,
            Payload."type" "txn_type",
            CAST(Payload.amount AS FLOAT) amount,
            Payload.nameOrig nameOrig,
            CAST(Payload.oldbalanceOrig AS FLOAT) oldbalanceOrig,
            CAST(Payload.newbalanceOrig AS FLOAT) newbalanceOrig,
            Payload.nameDest nameDest,
            CAST(Payload.oldbalanceDest AS FLOAT) oldbalanceDest,
            CAST(Payload.newbalanceDest AS FLOAT) newbalanceDest,
            CAST(Payload.isFraud AS INTEGER) isFraud,
            CAST(Payload.isFlaggedFraud AS INTEGER) isFlaggedFraud

        FROM (
            LOCATION = '/s3/s3.amazonaws.com/trial-datasets/FraudReduction/'
            AUTHORIZATION = repositories.PubAuth
           ) D

In [ ]:
-- Show Table Versioning for Time Travel capabilities
SELECT TableName, OFSStartVersionNumber, VersionTimeStamp FROM DBC.MultiVersionInfoTblV
WHERE DatabaseName = 'demo'
AND TableName = 'Txn_History_OFS'
ORDER BY 2;

In [ ]:
/* Count the number of rows after each version
Replace the timestamps as necessary 
Or use the version number */

SELECT 'Original' as Ver, COUNT(*) 
FROM demo.Txn_History_OFS 
    --AS OF TIMESTAMP '2023-02-13 11:19:28.360000-04:00'
    AS OF 0

GROUP BY 1

UNION ALL 

SELECT 'New Ver' as Ver, COUNT(*) 
FROM demo.Txn_History_OFS 
    --AS OF TIMESTAMP '2023-02-13 15:20:04.580000-04:00'
    AS OF 1

GROUP BY 1

<hr>
<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Demonstration 2 - Data in External Object Stores</b></p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '40%'>
            <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Vast quantities of data stored in Object Storage
                    <br>
                <ul>
                    <br>
                    <li>Data Lakes</li>
                    <br>
                    <li>IOT/IIOT</li>
                    <br>
                    <li>Marketplaces</li><br>
                    </ul></li>
                <br>
                <li>Vantage can read this data directly using built-in capabilities:
                    <br>
                    <ul><br>
                        <li>At massive scale/parallelism</li>
                    <br>
                        <li>Automatic schema discovery</li>
                        <br>
                            <li>Read optimizations/object filtering</li>
                            </ul></li>
                <br>
                <li>Administrators can create objects to enable easy access/analysis:
                <ul><br>
                    <li>Foreign Tables that contain object locations and definitions</li>
                    <br>
                    <li>Views to simplify access for users</li>
                    </ul>
                </li>
            </ul>
        </td>
        <td><img src = 'images/Data_Marketplace.png' width = '600'></td>
    </tr>
</table>

<hr>

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Exploration and Access Data in Object Stores</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The illustrations below review some common query types for investigating data in object storage.  Adding a "RETURNTYPE" parameter to the query can return additional information about the data:</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>NOSREAD_KEYS</b> will return objects keys/key paths, length in bytes, etc.</li>
    <li><b>NOSREAD_SCHEMA</b> will return the objects schema as interpreted by Vantage</li>
    </ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Data Engineer can then create a <b>Foreign Table</b> which acts as a virtual pointer to the data as it lies in Object Storage.  Optionally, this table can contain filtering mechanisms to assist in minimizing download volumes and improving performance.</p>

In [ ]:

SELECT TOP 5 *

FROM (
    LOCATION = '/s3/s3.amazonaws.com/td-usecases-data-store/retail_sample_data/FSCustomerJourney/ich_banking/'
    AUTHORIZATION = repositories.PubAuth
) AS D;

In [ ]:
/* Investigate object structure */

SELECT Location, ObjectLength

FROM (
    LOCATION = '/s3/s3.amazonaws.com/td-usecases-data-store/retail_sample_data/FSCustomerJourney/ich_banking/'
    AUTHORIZATION = repositories.PubAuth
    RETURNTYPE = 'NOSREAD_KEYS'
) AS D

In [ ]:
/* Investigate object schema */

SELECT TOP 12 *

FROM (
    LOCATION = '/s3/s3.amazonaws.com/td-usecases-data-store/retail_sample_data/FSCustomerJourney/ich_banking/'
    AUTHORIZATION = repositories.PubAuth
    RETURNTYPE = 'NOSREAD_SCHEMA'
) AS D
ORDER BY 1

In [ ]:
DROP TABLE demo.Banking_History_S3

In [ ]:
/* Create the Foreign Table */

CREATE FOREIGN TABLE demo.Banking_History_S3,
    EXTERNAL SECURITY repositories.PubAuth
    (
    customer_skey                 integer,
    customer_identifier           varchar(26),
    customer_cookie               varchar(50),
    customer_online_id            varchar(3),
    customer_offline_id           integer,
    customer_type                 varchar(90),
    customer_days_active          integer,
    interaction_session_number      bigint,
    interaction_timestamp         timestamp with time zone,
    interaction_source            varchar(100),
    interaction_type              varchar(100),
    sales_channel            varchar(100))
USING
(
     LOCATION  ('/s3/s3.amazonaws.com/td-usecases-data-store/retail_sample_data/FSCustomerJourney/ich_banking/')
     MANIFEST  ('FALSE')
     ROWFORMAT  ('{"field_delimiter":",","record_delimiter":"\n","character_set":"LATIN"}')
     STOREDAS  ('TEXTFILE')
     HEADER  ('FALSE')
     STRIP_EXTERIOR_SPACES  ('FALSE')
     STRIP_ENCLOSING_CHAR  ('"')
)
NO PRIMARY INDEX ;

In [ ]:
SELECT TOP 10 * FROM demo.Banking_History_S3

In [ ]:
SELECT COUNT(*) FROM demo.Banking_History_S3

<hr>
<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Demonstration 3 - Join data across storage types</b></p>

<table style = 'width:100%;table-layout:fixed;font-size:16px;font-family:Arial;color:#00233C'>
    <tr style = 'text-align:center'>
        <td><b>Customers_BFS, Accounts_Mapping_BFS</b>
        <br>Two dimension tables, stored in local BFS Storage on the Primary Cluster.</td>
                <td><b>Txn_History_OFS</b>
            <br>A large fact table with financial transaction detail information.  Data is stored in the Vantage Object Filesystem.</td>      
        <td><b>Banking_History_S3</b>
            <br>An extremely large external table containing Banking History. Data is stored as-is in S3.</td>
    </tr>
    </table>
    <table style = 'width:100%;table-layout:fixed;font-size:16px;font-family:Arial;color:#00233C'>
    <tr>
        <td><img src = 'images/EBS_Tables.png'></td>
        <td><img src = 'images/OFS_Tables.png'></td>
        <td><img src = 'images/S3_Tables.png'></td>
    </tr>
</table>

In [ ]:
/* Find banking users who have had a fradulent event in their cashapp account*/

SELECT TOP 10

txn.amount,
c.firstname,
c.lastname


FROM demo.txn_history_ofs txn, demo.Customers_BFS c, demo.Accounts_Mapping_BFS am


WHERE txn.isFraud = 1 AND
    txn.nameOrig= am.nameOrig AND
    am.customer_identifier = c.customer_identifier 

GROUP BY 1,2,3

<hr>

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Bringing it all together with ClearScape Analytics Functions</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This query will add two new elements to the analysis; adding in the large fact table stored natively in S3, and wrapping everything in a multi-row pattern analysis function named nPath. The goal of this analysis is identifying offline behavioral patterns leading to "Fraud" - a potentially valuable analysis that can directly feed more accurate Fraud prediction or early identification.</p>


<table style = 'width:100%;table-layout:fixed;font-size:16px;font-family:Arial;color:#00233C'>
    <tr style = 'text-align:top'>
        <td style = 'width:30%'>
        <ol>
            <br>
            <li>Use the Join conditions above to identify Fraud in mobile app</li>
            <br>
            <li>Join the fradulent users to their offline identities</li>
            <br>
            <li>Pass this data to Teradata nPath function to idenfify offline bevaviors</li>
            <br>
        </ol></td>
        <td><img src = 'images/Life_Of_Query.png'></td>
    </tr>
</table>

In [ ]:
/* Investigate user banking activities that preceeded a fraudulent online transaction */

SELECT * FROM NPATH
    (ON (
        SELECT
                bh.customer_identifier,
                bh.interaction_session_number,
                bh.interaction_timestamp,
                bh.interaction_type,
                txn.amount,
                c.firstname,
                c.lastname


                FROM demo.txn_history_ofs txn, demo.Banking_History_S3 bh, 
                    demo.Customers_BFS c, demo.Accounts_Mapping_BFS am


                WHERE txn.isFraud = 1 AND
                    txn.nameOrig= am.nameOrig AND
                    am.customer_identifier = bh.customer_identifier AND
                    am.customer_identifier = c.customer_identifier
        )
        
        PARTITION BY interaction_session_number
        ORDER BY interaction_timestamp
    USING
        MODE (NONOVERLAPPING)
        SYMBOLS (TRUE AS A)
        PATTERN ('A*')
        RESULT (FIRST (interaction_timestamp of A) as start_time, 
                FIRST (customer_identifier of A) as customer_identifier,
                FIRST (firstname of A) as firstname, 
                FIRST (lastname of A) as lastname,
                FIRST (amount of A) as fraud_amount,
                ACCUMULATE (interaction_type of ANY(A)) as path,
                FIRST (interaction_session_number of A) as interaction_session_number
                
    ));

<hr>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Seamlessly Join Data Across the Data Fabric</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>QueryGrid</b> provides seamless, high-performance data access with pushdown processing and parallel data movement across systems in heterogeneous analytical environments, including hybrid and multi-cloud.</p>

<table style = 'width:75%;table-layout:fixed;font-size:16px;font-family:Arial;color:#00233C'>
    <tr style = 'text-align:top'>
        <td style = 'width:50%'>
        <ol>
            <br>
            <li><b>Orchestrate</b> query execution across databases and platforms without migrating data</li>
            <br>
            <li><b>Launch</b> queries in place and execute pushdown processing across systems</li>
            <br>
            <li><b>Join</b> and transfer data across tables and systems from multiple sources</li>
            <br>
            <li><b>Reduce</b> data movement and minimize egress charges</li>
            <br>
        </ol></td>
        <td><img src = 'images/QG_Overview.png' width = '400' ></td>
    </tr>
</table>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For this demonstration, the <b>Transaction History</b> data set has been moved to an external RDBMS connected via VantageCloud Lake QueryGrid.  After executing this query, administrators can view detailed telemetry on data transfer rates, query pushdown, and other metrics in the <b>VantageCloud Lake Console.</b>  Alternatively, users can run an <b>EXPLAIN</b> to show the QueryGrid operations.</p>

In [ ]:
SELECT * FROM NPATH 
    (ON ( 
        SELECT 
                bh.customer_identifier, 
                bh.interaction_session_number, 
                bh.interaction_timestamp, 
                bh.interaction_type, 
                txn.amount, 
                c.firstname, 
                c.lastname 
 
 
                FROM demo_schema.txn_history@fs_postgres txn, demo.Banking_History_S3 bh,  
                    demo.Customers_BFS c, demo.Accounts_Mapping_BFS am 
 
 
                WHERE txn.isFraud = 1 AND 
                    txn.nameOrig= am.nameOrig AND 
                    am.customer_identifier = bh.customer_identifier AND 
                    am.customer_identifier = c.customer_identifier 
        ) 
         
        PARTITION BY interaction_session_number 
        ORDER BY interaction_timestamp 
    USING 
        MODE (NONOVERLAPPING) 
        SYMBOLS (TRUE AS A) 
        PATTERN ('A*') 
        RESULT (FIRST (interaction_timestamp of A) as start_time,  
                FIRST (customer_identifier of A) as customer_identifier, 
                FIRST (firstname of A) as firstname,  
                FIRST (lastname of A) as lastname, 
                FIRST (amount of A) as fraud_amount, 
                ACCUMULATE (interaction_type of ANY(A)) as path, 
                FIRST (interaction_session_number of A) as interaction_session_number )); 

In [ ]:
%disconnect lake_demo